# Object detection using contours

In [1]:
import cv2
import numpy as np

In [4]:
def nothing(x):
    pass

cv2.namedWindow("Object_detection")

cv2.createTrackbar("th","Object_detection",0,255, nothing)

cv2.createTrackbar("lb","Object_detection",0,255, nothing)
cv2.createTrackbar("lg","Object_detection",0,255, nothing)
cv2.createTrackbar("lr","Object_detection",0,255, nothing)

cv2.createTrackbar("hb","Object_detection",255,255, nothing)
cv2.createTrackbar("hg","Object_detection",255,255, nothing)
cv2.createTrackbar("hr","Object_detection",255,255, nothing)

cap = cv2.VideoCapture(0)
while cap.isOpened():
    r,frame = cap.read()
    if not r:
        break

    thr = cv2.getTrackbarPos("th","Object_detection")

    LB = cv2.getTrackbarPos("lb","Object_detection")
    LG = cv2.getTrackbarPos("lg","Object_detection")
    LR = cv2.getTrackbarPos("lr","Object_detection")

    HB = cv2.getTrackbarPos("hb","Object_detection")
    HG = cv2.getTrackbarPos("hg","Object_detection")
    HR = cv2.getTrackbarPos("hr","Object_detection")

    lower = np.array([LB,LG,LR])
    upper = np.array([HB,HG,HR])
        
    frame = cv2.flip(frame,1)
    frame = cv2.resize(frame,(300,300))
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    m = cv2.inRange(hsv,lower,upper)
    res = cv2.bitwise_and(frame,frame,mask = m)
    fr = cv2.bitwise_not(res)                     #It reduce the noise

    ret,thi = cv2.threshold(m,thr,255,cv2.THRESH_BINARY)
    cnt,hr = cv2.findContours(thi,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    cv2.drawContours(frame,cnt,-1,(255,0,0),2)

    cv2.imshow("thr",thi)
    cv2.imshow("res",res)
    cv2.imshow("mask",m)
    #cv2.imshow("hsv",hsv)
    cv2.imshow("live_frame",frame)
    
    if cv2.waitKey(25) & 0xff == ord("p"):
        break



cap.release()
cv2.destroyAllWindows()


# Image bg removal with histrogram

In [27]:
img = cv2.imread("C:/Users/sumit/OneDrive/Desktop/cvimg/green.jpeg") 
img = cv2.resize(img,(300,400))
hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

img1 = cv2.imread("C:/Users/sumit/OneDrive/Desktop/cvimg/green_bg.jpeg")
hsv2 = cv2.cvtColor(img1,cv2.COLOR_BGR2HSV)

hist = cv2.calcHist([hsv2],[0,1],None,[180,256],[0,180,0,256])
mask = cv2.calcBackProject([hsv],[0,1],hist,[0,180,0,256],1)

#res = cv2.bitwise_or(img,img,mask = mask)

ker = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
mask = cv2.filter2D(mask,-1,ker)

_,thr = cv2.threshold(mask,230,255,cv2.THRESH_BINARY)

mask = cv2.merge((mask,mask,mask))

res = cv2.bitwise_or(img,mask)



cv2.imshow("res",res)                   
cv2.imshow("org",img)
cv2.waitKey(0)
cv2.destroyAllWindows()